In [ ]:
import time
import numpy as np
from PIL import Image

import chainer
from chainer import configuration
import chainer.links as L
import chainer.functions as F
from chainer import serializers

from lib.ev3 import EV3
from lib.vstream import VideoStream


touch_port = EV3.PORT_2
color_port = EV3.PORT_3
lmotor_port = EV3.PORT_B
rmotor_port = EV3.PORT_C

SHIROMIGI=0
MONOMANE=1
IMETORE=2

# Network definition
class MLP(chainer.Chain):

    def __init__(self):
        super(MLP, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(300, 256)  # 300(20*15) -> 256 units
            self.l2 = L.Linear(256, 256)  # 256 units -> 256 units
            self.l3 = L.Linear(256, 1)      # 256 units -> 1

    def forward(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)


# Set up a neural networks of trained model
predictor_monomane = MLP()
predictor_imetore = MLP()

# Load the models
serializers.load_npz('ml_linetrace_model/monomane.model', predictor_monomane)
serializers.load_npz('ml_linetrace_model/imetore.model', predictor_imetore)

# Run VideoStream by setting image_size and fps
vs = VideoStream(resolution=(20, 15),
                 framerate=10,
                 colormode='binary').start()

ev3 = EV3()
ev3.enable_watchdog_task()
ev3.motor_config(lmotor_port, EV3.LARGE_MOTOR)
ev3.motor_config(rmotor_port, EV3.LARGE_MOTOR)
ev3.sensor_config(touch_port, EV3.TOUCH_SENSOR)
ev3.sensor_config(color_port, EV3.COLOR_SENSOR)

print("Push the touch sensor to start the linetracer")
while not ev3.touch_sensor_is_pressed(touch_port):
    pass

# Confirm the touch sensor is released.
while ev3.touch_sensor_is_pressed(touch_port):
    pass


mode = SHIROMIGI
ev3.lcd_draw_string('SHIROMIGI GO', 0)
# Enable evaluation mode for faster inference.
with configuration.using_config('train', False), chainer.using_config('enable_backprop', False):
    while True:
        # Break when the touch sensor is pressed.
        if ev3.touch_sensor_is_pressed(touch_port):
            break

        # Stop this loop when the down bottun was pressed.
        if ev3.button_is_pressed(EV3.DOWN_BUTTON):
            ev3.lcd_draw_string('STOPPING', 1)
            while ev3.button_is_pressed(EV3.DOWN_BUTTON):
                pass
            ev3.motor_steer(lmotor_port, rmotor_port, 0, 0)
            while not ev3.button_is_pressed(EV3.DOWN_BUTTON):
                time.sleep(0.01) 
            while ev3.button_is_pressed(EV3.DOWN_BUTTON):
                pass
            ev3.lcd_draw_string('', 1)

        # Change the mode by EV3's buttons
        if ev3.button_is_pressed(EV3.ENTER_BUTTON):
            mode = SHIROMIGI
            ev3.lcd_draw_string('SHIROMIGI GO', 0)
        if ev3.button_is_pressed(EV3.LEFT_BUTTON):
            mode = MONOMANE
            ev3.lcd_draw_string('MONOMANE GO', 0)
        if ev3.button_is_pressed(EV3.RIGHT_BUTTON):
            mode = IMETORE
            ev3.lcd_draw_string('IMETORE', 0)

        # Calculate a steer value by each mode
        if mode == SHIROMIGI:
            white = 70
            black = 5
            midpoint = (white + black) / 2
            color = ev3.color_sensor_get_reflect(color_port)
            steer = 1.4 * (color - midpoint)
        if mode == MONOMANE:
            im = vs.read()
            im = np.asarray(im, dtype=np.float32)
            x = im / 255.
            x = x.reshape(1, 300)
            y = predictor_monomane(x)
            steer = y.data[0, 0]
        if mode == IMETORE:
            im = vs.read()
            im = np.asarray(im, dtype=np.float32)
            x = im / 255.
            x = x.reshape(1, 300)
            y = predictor_imetore(x)
            steer = y.data[0, 0]
        print("steer = {}".format(steer))
        ev3.motor_steer(lmotor_port, rmotor_port, 10, int(steer))

vs.stop()
ev3.motor_steer(lmotor_port, rmotor_port, 0, 0)
ev3.close()